In [ ]:
import zarr
import xarray as xr
import numpy as np
import time
import math
from geopy.distance import geodesic

In [ ]:
st = time.time()
ds = xr.open_zarr(
    'GEBCO_2022_sub_ice_topo.zarr', chunks='auto', 
    group='gebco', decode_cf=False, decode_times=False)

et = time.time()
print('Exe time: ', et-st, 'sec')
ds

In [ ]:
# base=90 for latitude; base=180 for longitude
def gridded_arcsec(x, base=90, arc=3600/15):
    return ((int(x) + base)* arc + math.ceil((x-int(x))*arc))

arc = int(3600/15) #arc
print(gridded_arcsec(0, 180, arc))
print(gridded_arcsec(0, 90, arc))
print(gridded_arcsec(-90, 90, arc))
print(gridded_arcsec(90, 90, arc))
print(gridded_arcsec(-180, 180, arc))
print(gridded_arcsec(180, 180, arc))
print(gridded_arcsec(0.0015, 180, arc))
print(gridded_arcsec(-0.00207, 180, arc))
print(gridded_arcsec(-85.375, 90, arc))
print(gridded_arcsec(48.37918, 90, arc))


In [58]:
def curDist(loc, dis=np.empty(shape=[0, 1], dtype=float)):
    if len(loc) < 2:
        return 0.0
    lk = len(loc)-1
    return(np.append(dis, 
                     geodesic((loc[lk-1, 1], loc[lk-1, 0]),
                              (loc[lk, 1], loc[lk, 0])).km))

curDist(np.array([[121.53967066033685,25.070035818788032], [121.56499lonx[414543,25.06911681954908]]))

array([2.55682213])

In [86]:
arcsec = 15
arc = int(3600/arcsec) #15 arc-second
basex = 180 #-180 - 180 <==> 0 - 360, half is 180
basey = 90  #-90 - 90 <==> 0 - 180, half is 90
halfxidx = 180 * arc   #in netcdf, longitude length = 86400
halfyidx = 90 * arc    #in netcdf, latitude length = 43200
lonx = [121.53459759571552,121.54984716760636,121.54980816495167,123.04052581855191] 
#test bug (and solved)
#[121.53931677160625,121.56022155691544,121.56022155691544
#[121.94812284683387,121.96988566305195,121.97004166843372]
#[122.26981235231769,122.46580467738616,122.52322927209325] 
#[121.53967066033685,121.5649924414543] 
#[121.52789789212326,121.58410196573966,121.57161216694406,121.53726523073809,121.53664076385839,121.57723257849847] 
#[123, 123.33, 123.38]
latx = [25.063806258890185,25.064795994003052,25.064336474403152,26.055463893200404]
#[25.071087690867383,25.072501510151895,25.07186529331565]
#[24.858791163101777,24.860419861758118,24.85971173423322]  
#[24.53014834691002,24.484704345640385,24.43356023689588]  
#[25.070035818788032,25.06911681954908] 
#[25.054700205734235,25.033770970073878,24.993599440972766,24.982847101743943,24.942092803893317,24.930769789455212] 
#[23.782329893520448,23.769188442795453]
subsetFlag = True
zmode = "line" #"point"
format = "row"

for i in range(len(lonx)-1):
    print(i)

0
1
2


In [87]:
if (len(lonx) != len(latx)):
    print("Input Error: check longitude, latitude length not equal")
    #return
# May not have exact lon, lat in gridded lon-lat, so need to calculate index  
elif (len(lonx) == 1):
    st = time.time()
    lon0 = gridded_arcsec(lonx[0], basex, arc)
    lat0 = gridded_arcsec(latx[0], basey, arc)
    mlon0x = ds["lon"][lon0]
    mlat0x = ds["lat"][lat0]
    st1 = ds.sel(lon=mlon0x, lat=mlat0x)
    loc1 = [lonx[0], latx[0]]
    xt1 = np.array([st1['elevation'].values])
    et = time.time()
    print('Exe time: ', et-st, 'sec')
    print(xt1)
else:
    st = time.time()
    idx1 = np.empty(shape=[0, 2], dtype=np.int16)
    loc1 = np.empty(shape=[0, 2], dtype=float)
    dis1 = np.array([0.0])
    mlon0 = np.min(lonx)
    #mlon1 = np.max(lonx)
    mlat0 = np.min(latx)
    #mlat1 = np.max(latx)
    #ds_s1 = ds.sel(lon=slice(mlon0, mlon1), lat=slice(mlat0, mlat1))
    mlonbase = gridded_arcsec(mlon0, basex, arc) if subsetFlag else 0 #if do subsetting dataset, reference-0-x,y should be biased
    mlatbase = gridded_arcsec(mlat0, basey, arc) if subsetFlag else 0
    print(mlonbase, mlatbase)
    #mlonidx1 = gridded_arcsec(mlon1, basex, arc)
    #mlatidx1 = gridded_arcsec(mlat1, basey, arc)
    for i in range(len(lonx)-1):
        lonidx0 = gridded_arcsec(lonx[i], basex, arc)
        latidx0 = gridded_arcsec(latx[i], basey, arc)
        lonidx1 = gridded_arcsec(lonx[i+1], basex, arc)
        latidx1 = gridded_arcsec(latx[i+1], basey, arc)
        #ds_s1 = ds.sel(lon=slice(lonx[i], lonx[i+1]), lat=slice(latx[i], latx[i+1]))
        #index from gridded_arcsec will change is reference is ds_s1, not ds
        if zmode == 'point':
            idx1 = np.append(
                idx1, [[latidx0-mlatbase, lonidx0-mlonbase]], axis=0)
            loc1 = np.append(loc1, [[lonx[i], latx[i]]], axis=0)
            dist = geodesic((latx[i], lonx[i]),
                            (latx[i+1], lonx[i+1])).km
            dis1 = np.append(dis1, dist, axis=None)
            if i == len(lonx)-2:
                idx1 = np.append(
                    idx1, [[latidx1-mlatbase, lonidx1-mlonbase]], axis=0)
                loc1 = np.append(loc1, [[lonx[i+1], latx[i+1]]], axis=0) 

        elif lonidx0 == lonidx1 and latidx0 == latidx1:
            idx1 = np.append(idx1, [[latidx0-mlatbase, lonidx0-mlonbase]], axis=0)
            loc1 = np.append(loc1, [[lonx[i], latx[i]]], axis=0)
            if i >= 1: # and len(loc1) < 2:
                #dis1 = np.append(dis1, 0.0, axis=None) #to match the same length
                dis1 = curDist(loc1, dis1)
    
        else:        
            if lonx[i] == lonx[i+1]:
                stepi = -1 if latidx0 > latidx1 else 1
                rngi = range(latidx0, latidx1+stepi, stepi)
                leni = len(rngi)
                for k,y in enumerate(rngi):                  
                    locy0 = y/arc - basey
                    locy0i= int(locy0)
                    doty0i= locy0-locy0i-0.25/arc #a small bias to make sure it's in grid
                    locy1 = locy0i + doty0i
                    if (k<(leni-1)) or (stepi == 1 and locy1 < latx[i+1]) or (stepi == -1 and locy1 > latx[i+1]):
                        print("yi: ", y, locy0, locy0i, locy1)
                        idx1 = np.append(idx1, [[y-mlatbase, lonidx0-mlonbase]], axis=0)
                        loc1 = np.append(loc1, [[lonx[i], locy1]], axis=0)
                        if i>=1 or k>=1:
                            dis1 = curDist(loc1, dis1)
                            #print("dist: ", dist, " at ", loc1[lk-1, 1], " from ", loc1[lk, 1])                    
            elif latx[i] == latx[i+1]:
                stepi = -1 if lonidx0 > lonidx1 else 1
                rngi = range(lonidx0, lonidx1+stepi, stepi)
                leni = len(rngi)
                for k,x in enumerate(rngi):                    
                    locx0 = x/arc - basex
                    locx0i= int(locx0)
                    dotx0i= locx0-locx0i-0.25/arc #a small bias to make sure it's in grid
                    locx1 = locx0i + dotx0i
                    if (k<(leni-1)) or (stepi == 1 and locx1 < lonx[i+1]) or (stepi == -1 and locx1 > lonx[i+1]):
                        print("xi: ", x, locx1)
                        idx1 = np.append(idx1, [[latidx0-mlatbase, x-mlonbase]], axis=0)
                        loc1 = np.append(loc1, [[locx1, latx[i]]], axis=0)
                        if i>=1 or k>=1:
                            dis1 = curDist(loc1, dis1)

            else:
                m = (latx[i+1]-latx[i])/(lonx[i+1]-lonx[i])
                b = latx[i] - m * lonx[i] #y = mx + b
                print("m, b:", m ,b)
                if np.absolute(m) <= 1:
                    lidx0 = lonidx0
                    lidx1 = lonidx1
                else: 
                    lidx0 = latidx0
                    lidx1 = latidx1

                stepi = -1 if lidx0 > lidx1 else 1
                rngi = range(lidx0, lidx1+stepi, stepi)
                leni = len(rngi)
                for k,s in enumerate(rngi):
                    if k==0: #should consider internal node not repeated twice
                        idx1 = np.append(idx1, [[latidx0-mlatbase, lonidx0-mlonbase]], axis=0)
                        loc1 = np.append(loc1, [[lonx[i], latx[i]]], axis=0)
                        if i >= 1:
                            dis1 = curDist(loc1, dis1)

                    else:
                        if np.absolute(m) <= 1:
                            locx0 = s/arc - basex
                            locx0i= int(locx0)
                            dotx0i= locx0-locx0i-0.25/arc #a small bias to make sure it's in grid
                            locx1 = locx0i + dotx0i 
                            locy1 = m * locx1 + b 
                            if (k<(leni-1)) or (stepi == 1 and locx1 < lonx[i+1]) or (stepi == -1 and locx1 > lonx[i+1]):
                                y = gridded_arcsec(locy1, basey, arc)
                                idx1 = np.append(idx1, [[y-mlatbase, s-mlonbase]], axis=0)
                                print("line: ", s, locx0, dotx0i, locx1, locy1, y)
                                loc1 = np.append(loc1, [[locx1, locy1]], axis=0)
                                dis1 = curDist(loc1, dis1)

                        else:
                            locy0 = s/arc - basey
                            locy0i= int(locy0)
                            doty0i= locy0-locy0i-0.25/arc
                            locy1 = locy0i + doty0i 
                            locx1 = (locy1 - b)/m 
                            if (k<(leni-1)) or (stepi == 1 and locy1 < latx[i+1]) or (stepi == -1 and locy1 > latx[i+1]):
                                x = gridded_arcsec(locx1, basex, arc)
                                idx1 = np.append(idx1, [[s-mlatbase, x-mlonbase]], axis=0)
                                print("line: ", s, locy0, doty0i, locy1, locx1, x)
                                loc1 = np.append(loc1, [[locx1, locy1]], axis=0)
                                dis1 = curDist(loc1, dis1)
                        #print("ith-turn: ", i, k, len(idx1), len(loc1), len(dis1))

            if i == len(lonx)-2: #and lonidx1 != idx1[len(idx1)-1, 1] and latidx1 != idx1[len(idx1)-1, 0]:
                idx1 = np.append(
                    idx1, [[latidx1-mlatbase, lonidx1-mlonbase]], axis=0)
                loc1 = np.append(loc1, [[lonx[i+1], latx[i+1]]], axis=0)
                dis1 = curDist(loc1, dis1)
                    
    et = time.time()
    print('Exe time: ', et-st, 'sec')
    print(idx1)
    print(loc1)
    print(dis1)
    np.savetxt("simu/test_loc.csv", loc1, delimiter=",", fmt='%f') 
    np.savetxt("simu/test_dis.csv", dis1, delimiter=",", fmt='%f') 


72369 27616
m, b: 0.06490248512891315 17.175908845785813
line:  72370 121.54166666666669 0.5406250000000189 121.54062500000002 25.064197452407125 27616
line:  72371 121.54583333333335 0.5447916666666818 121.54479166666668 25.064467879428495 27616
line:  72372 121.55000000000001 0.5489583333333447 121.54895833333335 25.064738306449865 27616
m, b: 0.6648659566106117 -55.749993007023775
line:  72373 121.55416666666667 0.5531250000000075 121.55312500000001 25.066541725110483 27616
line:  72374 121.55833333333334 0.5572916666666704 121.55729166666667 25.069311999929695 27617
line:  72375 121.5625 0.5614583333333333 121.56145833333333 25.072082274748908 27618
line:  72376 121.56666666666666 0.5656249999999962 121.565625 25.074852549568106 27618
line:  72377 121.57083333333333 0.569791666666659 121.56979166666666 25.07762282438732 27619
line:  72378 121.57499999999999 0.5739583333333219 121.57395833333332 25.080393099206532 27620
line:  72379 121.57916666666665 0.5781249999999848 121.57812499

In [88]:
st = time.time()
#mlon0 = np.min(lonx) #may cause slice offset to mlonbase, an offset +-1
mlon1 = np.max(lonx) + 1.5/arc #to make it larger
#mlat0 = np.min(latx) #may cause slice offset to mlatbase, an offset +-1
mlat1 = np.max(latx) + 1.5/arc
mlon0x= ds["lon"][mlonbase]
mlat0x= ds["lat"][mlatbase]
ds_s1 = ds.sel(lon=slice(mlon0x, mlon1), lat=slice(mlat0x, mlat1)) if subsetFlag else ds
print(ds_s1['elevation'].shape)
#mlonidx0 = gridded_arcsec(mlon0, basex, arc)
#mlatidx0 = gridded_arcsec(mlat0, basey, arc)
#mlonidx1 = gridded_arcsec(mlon1, basex, arc)
#mlatidx1 = gridded_arcsec(mlat1, basey, arc)
loct = tuple(idx1.T)
print(loct)
pt1=ds_s1['elevation'].values[loct]
et = time.time()
print('Get index of points, time: ', et-st, 'sec')
print(pt1)
np.savetxt("simu/test_zseg.csv", pt1, delimiter=",", fmt='%f')

(239, 362)
(array([  0,   0,   0,   0,   0,   0,   0,   1,   2,   2,   3,   4,   4,
         5,   6,   6,   7,   8,   8,   9,  10,  10,  11,  12,  12,  13,
        14,  14,  15,  16,  16,  17,  18,  18,  19,  20,  20,  21,  22,
        22,  23,  24,  24,  25,  26,  26,  27,  28,  28,  29,  30,  30,
        31,  32,  32,  33,  34,  34,  35,  36,  36,  37,  38,  38,  39,
        40,  40,  41,  42,  42,  43,  44,  44,  45,  46,  46,  47,  48,
        48,  49,  50,  50,  51,  52,  52,  53,  54,  54,  55,  56,  56,
        57,  58,  58,  59,  60,  60,  61,  62,  62,  63,  64,  64,  65,
        66,  66,  67,  68,  68,  69,  70,  70,  71,  72,  72,  73,  74,
        74,  75,  76,  76,  77,  78,  78,  79,  80,  80,  81,  82,  82,
        83,  84,  84,  85,  86,  86,  87,  88,  88,  89,  90,  90,  91,
        92,  92,  93,  94,  94,  95,  96,  96,  97,  98,  98,  99, 100,
       100, 101, 102, 102, 103, 104, 104, 105, 106, 106, 107, 108, 108,
       109, 110, 110, 111, 112, 112, 113, 113, 114, 

In [ ]:
ds_s1 = ds.sel(lon=slice(mlon0, mlon1), lat=slice(mlat0, mlat1)) 
print(ds_s1)
print(ds["lat"][26720]) #actually more near 21.34, not 21.33 so slice 21.33 makes the offset 1 possibly!!
print(ds["lon"][72720]) #should use print(ds["lat"][26720]) to calcuate back
print(ds["elevation"].values[26720,72720]) 

In [ ]:
loc1[:,1]

In [95]:
from fastapi.encoders import jsonable_encoder
#from fastapi.responses import JSONResponse
json_data = jsonable_encoder({"longitude": loc1[:,0].tolist(), "latitude": loc1[:,1].tolist(),"z": pt1.tolist()})
#print(json_data)

import pandas as pd
from fastapi.responses import JSONResponse

st = time.time()
df1 = pd.DataFrame({"longitude": loc1[:,0].tolist(), "latitude": loc1[:,1].tolist(),"z": pt1.tolist()}, columns=['longitude', 'latitude', 'z'])
et = time.time()
print('Pandas time: ', et-st, 'sec')
print(df1)

import polars as pl
st = time.time()
df2 = pl.DataFrame({"longitude": loc1[:,0].tolist(), "latitude": loc1[:,1].tolist(),"z": pt1.tolist()}, columns=['longitude', 'latitude', 'z'])
et = time.time()
print('Polars time: ', et-st, 'sec')
print(df2)

#output time
st = time.time()
out1 = df1.to_dict(orient='records')
j1 = JSONResponse(content=out1)
et = time.time()
print('Output Pandas time: ', et-st, 'sec')

st = time.time()
out2 = df2.to_dicts()
#print(out2)
j2 = JSONResponse(content=out2)
et = time.time()
print('Output Polars time: ', et-st, 'sec')

print(j2)

Pandas time:  0.002172708511352539 sec
      longitude   latitude    z
0    121.534598  25.063806    7
1    121.540625  25.064197    7
2    121.544792  25.064468    5
3    121.548958  25.064738    3
4    121.549847  25.064796    3
..          ...        ...  ...
359  123.023958  26.044449 -101
360  123.028125  26.047219 -102
361  123.032292  26.049989 -102
362  123.036458  26.052760 -102
363  123.040526  26.055464 -102

[364 rows x 3 columns]
Polars time:  0.0002353191375732422 sec
shape: (364, 3)
┌────────────┬───────────┬──────┐
│ longitude  ┆ latitude  ┆ z    │
│ ---        ┆ ---       ┆ ---  │
│ f64        ┆ f64       ┆ i64  │
╞════════════╪═══════════╪══════╡
│ 121.534598 ┆ 25.063806 ┆ 7    │
├╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌┤
│ 121.540625 ┆ 25.064197 ┆ 7    │
├╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌┤
│ 121.544792 ┆ 25.064468 ┆ 5    │
├╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌┤
│ 121.548958 ┆ 25.064738 ┆ 3    │
├╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌┤
│ ...        ┆ ...       ┆ ...  │
├╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌

In [ ]:
from fastapi.encoders import jsonable_encoder
#just some  simple testing for query string
lax = "25.5, 30.22,20"
lox = " 125.33"
#la1 = lax.split(",")
if (',' in lax):
    try:
        la1 = [float(x.strip()) for x in lax.split(',')]
    except ValueError:
        print("Check your input format") 
else: 
    la1 = [float(lax.strip())]

if (',' in lox):
    lo1 = lox.split(",")
else: 
    lo1 = [float(lox.strip())]
print(la1)
print(lo1)

lon0 = gridded_arcsec(123, basex, arc)
lat0 = gridded_arcsec(21.33, basey, arc)
mlon0x = ds["lon"][lon0]
mlat0x = ds["lat"][lat0]
st1 = ds.sel(lon=mlon0x, lat=mlat0x)
xt1 = np.array([st1['elevation'].values])
print(xt1)
jt1 = {"data": xt1.tolist()}
jsonable_encoder(jt1)

In [ ]:
dis1 = np.array([0])
st = time.time()
for i in range(len(loc1)-1):
  dist = geodesic((loc1[i,1],loc1[i,0]), (loc1[i+1,1],loc1[i+1,0])).km
  dis1 = np.append(dis1, dist, axis=None)
et = time.time()
print('Calculate loop of geo dist time: ', et-st, 'sec')
print(dis1)

In [ ]:
#https://stackoverflow.com/questions/72620476/faster-way-to-calculate-distance-from-coordinates-on-dataframe
st = time.time()
df2 = pd.merge(df1.rename(columns={'latitude':'clat','longitude':'clon'}), df1, how='cross') \
            .assign(distance=lambda r: r.apply(lambda x: geodesic((x['clat'],x['clon']),(x['latitude'],x['longitude'])).km, axis=1)
            ) #.drop(columns=['clat','clon'])
et = time.time()
print('Calculate df of geo dist time: ', et-st, 'sec')
print(df2)

In [72]:
# want to solve a situation when cross 180-line or zero-line (prime meridian)
l1a = [-179.95, 25]
l2a = [179,95, 24]
l1b = [-0.05, 25]
l2b = [0.05, 24]

print(geodesic((25,-179.95), (25,179.95)).km)
print(geodesic((25,-0.05), (25,0.05)).km)

def whichSide(l1a, l2a, direction='auto'):
   #posFlag = 'near-zero' #near prime meridian (near-zero) #otherwise, far-zero (near 180-degree)
   #autoFly: automatic get the shortest direction to cross prime meridian or 180-degree line
   if l1a[0] != l2a[0] and np.sign(l1a[0]) != np.sign(l2a[0]):
      lonat1 = 180 + l1a[0]
      lonat2 = 180 + l2a[0]
      dift1 = np.absolute(lonat2 - lonat1)

      lonas1 = 180 - l1a[0] if l1a[0]>=0 else 180 - l2a[0]
      lonas2 = -180- l2a[0] if l2a[0]<0 else -180 - l1a[0]
      dift2 = np.absolute(lonas2 - lonas1) 

      if dift1 < dift2:
         posFlag = 'near-zero'
         autoFly = 'forward'
      else:
         posFlag = 'far-zero'
         autoFly = 'backward'

      print("AutoFly: ", autoFly, posFlag)
      return(autoFly, posFlag)      

whichSide(l1a, l2a)
whichSide(l1b, l2b)
whichSide([-35.95, 25], [35, 24])
whichSide([-130.95, 25], [35, 24])
tt = whichSide([-160.95, 25], [35, 24])
print(tt[0], tt[1])



10.095008799533332
10.095008799531035
AutoFly:  backward far-zero
AutoFly:  forward near-zero
AutoFly:  forward near-zero
AutoFly:  forward near-zero
AutoFly:  backward far-zero
backward far-zero
